В данном задании нужно было попрактиковаться в работе с синтаксическим парсером SyntaxNet и в извлечении SVO-троек из предложений. Необходимо было улучшить простейший код, извлекающий SVO-тройки, чтобы они были полными, чтобы учитывался порядок слов в предложении и однородные члены, после чего придумать собственные примеры и протестировать его на них.

In [39]:
from nltk import DependencyGraph
import codecs

def svo(file, tree=False):
    processed_sentences = []
    sentence = []
    for line in codecs.open(file, 'r', 'utf-8'):
        if len(line) == 1:
            processed_sentences.append(sentence)
            sentence = []
        else:
            word = line.split("\t")
            sentence.append(word)
    
    deps = []
    for sentence in processed_sentences:
        s = u''
        for line in sentence:
            s += u"\t".join(line) + u'\n'
        deps.append(s)
    
    if tree:
        for sent_dep in deps:
            # борьба с кодировкой
            sent_dep = sent_dep.replace('\ufeff', '')
            graph = DependencyGraph(tree_str=sent_dep)
            for triple in graph.triples():
                for elem in triple:
                    print (elem[0] if isinstance(elem, tuple) else elem,)
                print()
            print()
            print(graph.tree().pretty_print())

    for sent_dep in deps:
        # борьба с кодировкой
        sent_dep = sent_dep.replace('\ufeff', '')
        verbs = {}
        for t in sent_dep.split('\n'):
            if len(t) > 1:
                splt = t.split('\t')
                if splt[3] == 'VERB':
                    # вместе с глаголом сохраним словарь, в который будем добавлять слова с соответствующими связями с ним
                    verbs[splt[0]] = (splt[1], {'nsubj': [], 'dobj': []})

        for t in sent_dep.split('\n'):
            if len(t) > 1:
                splt = t.split('\t')
                if splt[7] in ['nsubj', 'dobj']:
                    if splt[6] in verbs:
                        # на основании связи с глаголом добавляем слово в словарь, сохраняя при этом его номер, чтобы учесть порядок
                        verbs[splt[6]][1][splt[7]].append((splt[1], splt[0]))

        # проходим по сохраненным номерам, обозначающим положение глагола в предложении
        for index in verbs.keys():
            # выделяем глагол
            verb = verbs[index][0]
            # выделяем все слова, имеющие связи nsubj и dobj с рассматриваемым глаголом
            nsubj = verbs[index][1]['nsubj']
            dobj = verbs[index][1]['dobj']
            # проверяем, что SVO-тройка полна
            if nsubj and dobj:
                # массив из троек (слово, связанное с глаголом связью nsubj; глагол; слово, связанное с глаголом связью dobj)
                triples = []
                # перечисляем все возможные тройки
                for nsubj_word in nsubj:
                    for dobj_word in dobj:
                        # добавляем тройку в массив из троек
                        triples.append((nsubj_word, (verb, index), dobj_word))
                # сортируем слова по положению в соответствующем предложении, чтобы сохранить порядок, и выводим
                for tr in triples:
                    print([a for (a, b) in sorted(tr, key=lambda x: x[1])])

In [40]:
svo('data.conll')

['Совет', 'продлил', 'санкции']
['Радикалы', 'зажгли', 'файеры']


Итак, на исходном примере все работает, как нужно. Рассмотрим следующие предложения:

> Я купил яблоки и бананы .

> Сумку и куртку ты оставил в коридоре .

> Утюгом я глажу бельё .

> Шлю я тебе большой привет и поцелуй .

Извлечены должны быть следующие SVO:

> Я купил яблоки, я купил бананы, Cумку ты оставил, куртку ты оставил, Утюгом я глажу, я глажу бельё, Шлю я тебе, Шлю я привет, Шлю я поцелуй

In [43]:
svo('data2.conll')

['Я', 'купил', 'яблоки']
['Сумку', 'ты', 'оставил']
['я', 'глажу', 'бельё']
['Шлю', 'я', 'привет']


В чем же проблема?

In [4]:
svo('data2.conll', tree=True)

купил
nsubj
Я

купил
dobj
яблоки

купил
cc
и

купил
conj
бананы

купил
punct
.


           купил           
  ___________|___________   
 Я  яблоки   и   бананы  . 

None
оставил
dobj
Сумку

Сумку
cc
и

Сумку
conj
куртку

оставил
nsubj
ты

оставил
nmod
коридоре

коридоре
case
в

оставил
punct
.


        оставил                      
  _________|____________________      
 |   |          Сумку        коридоре
 |   |      ______|_____        |     
 ты  .     и          куртку    в    

None
глажу
nmod
Утюгом

глажу
nsubj
я

глажу
dobj
бельё

глажу
punct
.


       глажу          
   ______|__________   
Утюгом   я   бельё  . 

None
Шлю
nsubj
я

Шлю
nmod
тебе

Шлю
dobj
привет

привет
amod
большой

привет
cc
и

привет
conj
поцелуй

Шлю
punct
.


         Шлю                       
  ________|____________             
 |   |    |          привет        
 |   |    |      ______|_______     
 я  тебе  .  большой   и    поцелуй

None
['Я', 'купил', 'яблоки']
['Сумку', 'ты', 'оставил']
['я',

In [7]:
processed_sentences = []
sentence = []
for line in codecs.open('data2.conll', 'r', 'utf-8'):
    if len(line) == 1:
        processed_sentences.append(sentence)
        sentence = []
    else:
        word = line.split("\t")
        sentence.append(word)

deps = []
for sentence in processed_sentences:
    s = u''
    for line in sentence:
        s += u"\t".join(line) + u'\n'
    deps.append(s)
    
deps[0].split('\n')

['1\tЯ\t_\tPRON\t_\tfPOS=PRON++\t2\tnsubj\t_\t_',
 '',
 '2\tкупил\t_\tVERB\t_\tAspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act|fPOS=VERB++\t0\tROOT\t_\t_',
 '',
 '3\tяблоки\t_\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Plur|fPOS=NOUN++\t2\tdobj\t_\t_',
 '',
 '4\tи\t_\tCONJ\t_\tfPOS=CONJ++\t2\tcc\t_\t_',
 '',
 '5\tбананы\t_\tVERB\t_\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Plur|fPOS=NOUN++\t2\tconj\t_\t_',
 '',
 '6\t.\t_\tPUNCT\t.\tfPOS=PUNCT++.\t2\tpunct\t_\t_',
 '',
 '']

Отлично, "бананы" - это, оказывается, глагол!

Видно, что в некоторых местах парсер выдал абсурдный результат. В таком случае логично немного отредактировать файл 'data2.conll', чтобы избавиться от самого бреда.

In [45]:
svo('data2_1.conll')

['Я', 'купил', 'яблоки']
['Я', 'купил', 'бананы']
['Сумку', 'ты', 'оставил']
['куртку', 'ты', 'оставил']
['Утюгом', 'я', 'глажу']
['я', 'глажу', 'бельё']
['Шлю', 'я', 'тебе']
['Шлю', 'я', 'привет']
['Шлю', 'я', 'поцелуй']


Так-то лучше.